https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d
https://github.com/slundberg/shap
https://shap.readthedocs.io/en/latest/index.html
https://github.com/slundberg/shap/blob/master/README.md
    
https://pypi.org/project/BorutaShap/
https://github.com/scikit-learn-contrib/boruta_py
https://github.com/Ekeany/Boruta-Shap/tree/master/Examples
    
https://github.com/slundberg/shap/issues/1120
https://github.com/slundberg/shap/issues/977
    
https://christophm.github.io/interpretable-ml-book/shapley.html#disadvantages-13
    
https://stats.stackexchange.com/questions/510510/intuitive-explanation-of-friedmans-h-statistic

https://www.investopedia.com/terms/v/variance-inflation-factor.asp

https://github.com/slundberg/shap/issues/1098

https://pacmedhealth.medium.com/explainability-for-tree-based-models-which-shap-approximation-is-best-6df78bc5d086

**SHAP values (SHapley Additive exPlanations)** is a method based on cooperative game theory and used to increase transparency and interpretability of machine learning models. More specifically, SHAP method is used to explain how each feature affects the model, and allows local and global analysis for the dataset and problem at hand.

**Important**: while SHAP shows the contribution or the importance of each feature on the prediction of the model, it does not evaluate the quality of the prediction itself.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor 
import shap
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

FEATURES = ['commits_count','code_churn_avg','contributors_count','contributors_experience','hunks_count',
            'issue_tracker_issues','cbo','wmc','dit','rfc','lcom','max_nested_blocks','total_refactorings',
            'ncloc_cloc','duplicated_lines_cpd_density','comment_lines_cloc_density','total_variables',
            'total_methods','Max-Ruler']

# Importing the dataset
dataset = pd.read_csv('dataset_before_preprocessing.csv', sep=',') # usecols = FEATURES

In [ ]:
# Java project names
projects_java = ['arduino','arthas','azkaban','cayenne','deltaspike',\
                 'exoplayer','fop','gson','javacv','jclouds','joda-time',\
                 'libgdx','maven','mina','nacos','opennlp','openrefine',\
                 'pdfbox','redisson','RxJava','testng','vassonic','wss4j',\
                 'xxl-job','zaproxy']

dataset = dataset.loc[dataset['project_name'] == 'arduino']

dataset = dataset[FEATURES]

In [ ]:
# Descriptive Statistic
dataset.describe()

## Remove metrics with multicollinearity based on VIF

SHAP is a post-hoc model-agnostic interpretability method that uses Shapley values from game theory to estimate the predictive importance (i.e. SHAP score) of the features of a machine learning model. When using SHAP, remember that we are explaining the machine learning model (rather than the data!!!). Correlation bias occurs because of how the machine learning algorithm trains the model, not because of how SHAP estimates feature importance. For example, when presented with a group of highly correlated features {A,B,C}, the machine learning algorithm will assign a large weight to one arbitrary representative of the group, let’s say A. Indeed, features B and C hardly provide any more information than A does (they are very similarly associated to the prediction target labels). Features B and C are redundant. The model’s decision-making process will therefore rely heavily on feature A, but not on features B and C. Consequently, features B and C will score poorly when SHAP is used to explain the model. If SHAP were to assign high importance scores to features B and C, it would not be faithful to the model.
https://github.com/slundberg/shap/issues/1120

In [ ]:
# All features
# X = dataset[['commits_count','code_churn_avg','contributors_count','contributors_experience','hunks_count',
#             'issue_tracker_issues','cbo','wmc','dit','rfc','lcom','max_nested_blocks','total_refactorings',
#             'ncloc_cloc','duplicated_lines_cpd_density','comment_lines_cloc_density','total_variables',
#             'total_methods']]

# Features after accounting for multicollinearity (‘wmc’, ‘ncloc’, ‘contributors_experience’, ‘rfc’, and ‘contributors_count’ are removed)
# X = dataset[['commits_count','code_churn_avg','hunks_count',
#             'issue_tracker_issues','cbo','dit','lcom','max_nested_blocks','total_refactorings',
#             'duplicated_lines_cpd_density','comment_lines_cloc_density','total_variables',
#             'total_methods']]

# Features after accounting for multicollinearity (‘commits_count’, ‘contributors_experience’, ‘rfc’, ‘ncloc_cloc’, 'total_variables' and ‘total_methods’ are removed)
X = dataset[['code_churn_avg','contributors_count','hunks_count',
            'issue_tracker_issues','cbo','wmc','dit','lcom','max_nested_blocks','total_refactorings',
            'duplicated_lines_cpd_density','comment_lines_cloc_density']]

vif_data = pd.DataFrame()
vif_data['metric'] = X.columns

# calculating VIF for each feature
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

# Plot Spearman correlation among features
corr = X.corr(method = 'spearman')
fig, ax = plt.subplots(figsize=(14,12))
ax = sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 10, as_cmap=True));

vif_data

## Build the project-specific classifiers

In [ ]:
# create X and Y
Y = dataset['Max-Ruler']

# All features
# X = dataset[['commits_count','code_churn_avg','contributors_count','contributors_experience','hunks_count',
#             'issue_tracker_issues','cbo','wmc','dit','rfc','lcom','max_nested_blocks','total_refactorings',
#             'ncloc_cloc','duplicated_lines_cpd_density','comment_lines_cloc_density','total_variables',
#             'total_methods']]

# Features after accounting for multicollinearity (‘wmc’, ‘ncloc’, ‘contributors_experience’, ‘rfc’, and ‘contributors_count’ are removed)
# X = dataset[['commits_count','code_churn_avg','hunks_count',
#             'issue_tracker_issues','cbo','dit','lcom','max_nested_blocks','total_refactorings',
#             'duplicated_lines_cpd_density','comment_lines_cloc_density','total_variables',
#             'total_methods']]

# Features after accounting for multicollinearity (‘commits_count’, ‘contributors_experience’, ‘rfc’, ‘ncloc_cloc’, 'total_variables' and ‘total_methods’ are removed)
X = dataset[['code_churn_avg','contributors_count','hunks_count',
            'issue_tracker_issues','cbo','wmc','dit','lcom','max_nested_blocks','total_refactorings',
            'duplicated_lines_cpd_density','comment_lines_cloc_density']]

weight_ratio = float(len(Y[Y == 0]))/float(len(Y[Y == 1]))

# # Arduino
model = RandomForestClassifier(n_estimators=200, criterion='entropy', max_depth=1, min_samples_leaf=1, min_samples_split=2, class_weight='balanced', random_state=0)
model.fit(X, Y)

# # Arthas
# model = RandomForestClassifier(n_estimators=600, criterion='gini', max_depth=4, min_samples_leaf=4, min_samples_split=10, class_weight='balanced', random_state=0)
# model.fit(X, Y)

# # azkaban
# model = XGBClassifier(n_estimators=100, max_depth=1, scale_pos_weight=weight_ratio, random_state=0)
# model.fit(X, Y)

# # cayenne
# model = XGBClassifier(n_estimators=100, max_depth=1, scale_pos_weight=weight_ratio, random_state=0)
# model.fit(X, Y)

# # deltaspike
# model = XGBClassifier(n_estimators=50, max_depth=1, scale_pos_weight=weight_ratio, random_state=0)
# model.fit(X, Y)

# # exoplayer
# model = XGBClassifier(n_estimators=20, max_depth=2, scale_pos_weight=weight_ratio, random_state=0)
# model.fit(X, Y)

# # fop
# model = XGBClassifier(n_estimators=200, max_depth=1, scale_pos_weight=weight_ratio, random_state=0)
# model.fit(X, Y)

# # jclouds
# model = RandomForestClassifier(n_estimators=200, criterion='entropy', max_depth=None, min_samples_leaf=4, min_samples_split=10, class_weight='balanced', random_state=0)
# model.fit(X, Y)

# # joda-time
# model = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=2, min_samples_leaf=1, min_samples_split=2, class_weight='balanced', random_state=0)
# model.fit(X, Y)

# # libgdx
# model = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=4, min_samples_leaf=4, min_samples_split=10, class_weight='balanced', random_state=0)
# model.fit(X, Y)

# # maven
# model = RandomForestClassifier(n_estimators=200, criterion='entropy', max_depth=None, min_samples_leaf=4, min_samples_split=2, class_weight='balanced', random_state=0)
# model.fit(X, Y)

# # mina
# model = RandomForestClassifier(n_estimators=200, criterion='gini', max_depth=4, min_samples_leaf=4, min_samples_split=2, class_weight='balanced', random_state=0)
# model.fit(X, Y)

# # nacos
# model = RandomForestClassifier(n_estimators=400, criterion='gini', max_depth=2, min_samples_leaf=1, min_samples_split=2, class_weight='balanced', random_state=0)
# model.fit(X, Y)

# # opennlp
# model = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=4, min_samples_leaf=1, min_samples_split=5, class_weight='balanced', random_state=0)
# model.fit(X, Y)

# # openrefine
# model = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=4, min_samples_leaf=4, min_samples_split=10, class_weight='balanced', random_state=0)
# model.fit(X, Y)

# # pdfbox
# model = XGBClassifier(n_estimators=100, max_depth=1, scale_pos_weight=weight_ratio, random_state=0)
# model.fit(X, Y)

# # redisson
# model = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=4, min_samples_leaf=4, min_samples_split=10, class_weight='balanced', random_state=0)
# model.fit(X, Y)

# # RxJava
# model = XGBClassifier(n_estimators=50, max_depth=2, scale_pos_weight=weight_ratio, random_state=0)
# model.fit(X, Y)

# # testng
# model = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=1, min_samples_leaf=1, min_samples_split=2, class_weight='balanced', random_state=0)
# model.fit(X, Y)

# # wss4j
# model = XGBClassifier(n_estimators=50, max_depth=2, scale_pos_weight=weight_ratio, random_state=0)
# model.fit(X, Y)

# # xxl-job
# model = XGBClassifier(n_estimators=50, max_depth=1, scale_pos_weight=weight_ratio, random_state=0)
# model.fit(X, Y)

# # zaproxy
# model = XGBClassifier(n_estimators=200, max_depth=1, scale_pos_weight=weight_ratio, random_state=0)
# model.fit(X, Y)

## Calculate SHAP values and SHAP interaction values using the model

In [ ]:
# Explain the model's predictions using SHAP
# explainer = shap.explainers.Tree(model, feature_perturbation='interventional', data=X, model_output='probability')
explainer = shap.explainers.Tree(model, feature_perturbation='tree_path_dependent')
shap_values = explainer.shap_values(X)
shap_interaction = explainer.shap_interaction_values(X)

In [ ]:
vals= np.abs(shap_values[1]).mean(0)
feature_importance = pd.DataFrame(list(zip(X.columns,vals)), columns=['metric','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'], ascending=False, inplace=True)
feature_importance

## SHAP summary plot

In [ ]:
# Create a SHAP summary plot, colored by feature values when they are provided.
# plot_type can be 'dot' (default for single output), 'bar' (default for multi-output), or 'violin'
# shap_values is multi-output and shap_values[1] is single output
shap.summary_plot(shap_values, features=X, plot_type='bar')

The above *SHAP summary plot* shows the predictors importance, i.e., it lists the most significant predictors (metrics) in descending order. The top predictors contribute more to the model than the bottom ones and thus have higher predictive power.

In [ ]:
shap.summary_plot(shap_values[1], features=X, plot_type='dot')

The above *SHAP bee swarm* plot can further show the positive and negative relations of the predictors (metrics) with the target variable (high-TD/not high-TD class). It can also show how higher and lower values of the predictors will affect the result. This plot is made of all the samples in the data, depicted as dots. It demonstrates the following information:
- **Feature importance**: Predictors are ranked in descending order.
- **Impact**: The horizontal location of each dot (SHAP value) shows whether the value of the associated predictor contributes towards a higher (high-TD) or lower (not high-TD) prediction for that observation. The further these dots extend on the X-axis (either positively or negatively), the higher their SHAP value and therefore their contribution to that prediction.
- **Original value**: Color of each dot shows whether the predictor’s value is high (in red) or low (in blue) for that observation.
- **Correlations**: As an example, higher values (red colored dots) of the “wmc” metric have a high and positive impact (X-axis) on labeling a class as high-TD. In addition, lower values (blue colored dots) of the “comment lines density” metric have a moderate and positive impact (X-axis) on labeling a class as high-TD. On the other hand, higher values (red colored dots) of the “comment lines density” metric have a moderate and negative impact (X-axis) on labeling a class as high-TD (i.e., positive on labeling a class as not-high-TD).

In [ ]:
shap.summary_plot(shap_interaction[1], features=X, plot_type='dot', max_display=15)

In [ ]:
mean_shap = np.abs(shap_interaction[1]).mean(0)
df = pd.DataFrame(mean_shap, index=X.columns, columns=X.columns)
df

## SHAP Thresholds

In [ ]:
threshold_df = pd.DataFrame()
for metric in list(X):
    metric_idx = X.columns.get_loc(metric)
    metric_shap_values = shap_values[1][:,metric_idx]
    index_thresh = np.argmin(np.absolute(metric_shap_values))
    X[metric].iloc[index_thresh]
    temp_df = pd.DataFrame([{'Metric':metric, 'Threshold':X[metric].iloc[index_thresh]}])    
    threshold_df = pd.concat([threshold_df, temp_df], ignore_index=True)
threshold_df

## SHAP dependence plot

In [ ]:
# Plots the value of the feature on the x-axis and the SHAP value of the same feature on the y-axis.
# This shows how the model depends on the given feature, and is like a richer extenstion of the 
# classical parital dependence plots. Vertical dispersion of the data points represents interaction effects. 
# interaction_index can be 'auto' (the strongest interaction this feature has with another feature), string, or None
shap.dependence_plot('wmc', shap_values[1], features=X, interaction_index='wmc')

The *SHAP dependence plot* shows the marginal effect one or two predictors (metrics) have on the predicted outcome of a ML model. It tells whether the relationship between the target and a predictor is linear, monotonic or more complex by plotting the value of the predictor on the x-axis and the SHAP value of the same predictor on the y-axis. SHAP value for a predictor represents how much knowing that predictor's value changes the output of the model for that sample's prediction. The function automatically includes another predictor that the chosen predictor interacts most with.

As an example, the above *SHAP dependence plot* shows that there is an approximately linear and positive trend between “wmc” and the target variable (high-TD), and how “wmc” interacts with “duplicated lines density”. In this example we can see that as weighted methods complexity (wmc) in a class increases from ~30 to ~100, the effect of this particular metric on labeling the class as high-TD increases significantly. In addition, weighted methods complexity (wmc) below ~30 in a class contribute towards labeling this class as not high-TD.

In [ ]:
shap.dependence_plot('comment_lines_cloc_density', shap_values[1], features=X, interaction_index='comment_lines_cloc_density')

In another example, the above *SHAP dependence plot* shows that when the density of comment lines in a class is below ~30%, this metric contributes towards labeling this class as high-TD. On the other hand, when the comment lines density is above ~30%, this metric contributes towards labeling a class as not high-TD.

In [ ]:
shap.dependence_plot('cbo', shap_values[1], features=X, interaction_index='cbo')

In another example, the above *SHAP dependence plot* shows that when the CBO in a class is below ~10, the effect of this metric is mixed, since it contributes towards both labeling a class as high-TD and as not high-TD. However, when the CBO in a class goes above ~10, it seems that this metric contributes only towards labeling a class as high-TD.

In [ ]:
shap.dependence_plot('max_nested_blocks', shap_values[1], features=X, interaction_index='auto')

In another example, the above *SHAP dependence plot* shows that when the nested blocks in a class are less than ~3, this metric contributes towards labeling a class as not high-TD. On the other hand, when the nested blocks in a class are more than ~3, this metric contributes towards labeling a class as high-TD.

In [ ]:
shap.dependence_plot('contributors_count', shap_values[1], features=X, interaction_index='auto')

Finally, in another example, the above *SHAP dependence plot* shows that when the contributors in a class are less than ~5, this metric contributes towards labeling a class as not high-TD. On the other hand, when the contributors in a class are more than ~5, this metric contributes towards labeling a class as high-TD.

## SHAP force plot (global interpratation)

In [ ]:
shap.initjs()

# This plot is a summary of n horizontally stacked individual plots, where n is the number of samples in the dataset 
shap.force_plot(explainer.expected_value[1], shap_values[1], features=X)

Regarding the weighted methods complexity (wmc) in a class, the same observations can be made by exploiting the collective *SHAP force plot* presented above. Each observation has its force plot. If all the force plots are combined, rotated 90 degrees, and stacked horizontally, we get the force plot of the entire dataset. The blue bands shows how much a feature draggs the final output value down (to 0 class), and the pink bands are those that increase it (up to 1 class). To understand how a single feature effects the output of the model we can plot the SHAP value of that feature vs. the value of the feature for all the examples in a dataset. We can clearly see that as wmc in a class increases beyond the value of ~30, the effect of this particular metric on labeling the class as high-TD increases significantly until the value of ~100, where it becomes constant but remains high. On the other hand, when the wmc is below 30, this metric contributes towards labeling a class as not high-TD.

Regarding the density of comment lines in a class, the same observations can be made by exploiting the *SHAP force plot* presented above. We can clearly see that when the density of comment lines in a class is below ~30%, this metric contributes towards labeling this class as high-TD. On the other hand, when the comment lines density is above ~30%, this metric contributes towards labeling this class as not high-TD.

Finally, regarding cbo, the same observations can be made by exploiting the *SHAP force plot* presented above. We can see that when the cbo in a class is above ~10, the effect of this particular metric on labeling the class as high-TD increases significantly until the value of ~40, where it becomes constant but remains high. On the other hand, when the cbo is below 10, this metric contributes towards labeling a class as not high-TD.

## SHAP force plot (local interpratation)

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][9,:], features=X.iloc[9,:])

Finally, *SHAP force plots* can be also used to examine the SHAP values on individual predictions. As an example, the above force plot shows how predictors (metrics) contributed to the model’s prediction for the first observation of the dataset. It demonstrates the following information:
- The f(x) value is the prediction for that observation (the prediction of the 7th row of the dataset).
- The base value is “the value that would be predicted if we did not know any features for the current output.” In other words, it is the mean of all predictions. It has a value of 0.067 because the mean prediction of the dataset is 0.067.
- Red/blue: Predictors that push the prediction higher (to the right – towards high-TD) are shown in red, while those pushing the prediction lower (to the left– towards not high-TD) are shown in blue.

In this example, “total refactorings”, “hunks count”, and "comment lines density" (among others) have a positive impact on labeling the class as high-TD. The number of refactorings applied to this class is 59, a value much higher than the average of our dataset (27.31 according to Table 4 of the TSE paper). We saw that higher values of the “total refactorings” metric have a positive impact on labeling a class as high-TD. Therefore, the value of this metric pushes the prediction to the right. Moreover, the median number of hunks made to this class in the evolution period is 4, a value higher than the average of our dataset (1.76 according to Table 4 of the TSE paper). We saw that higher values of the “hunks count” metric have a positive impact on labeling a class as high-TD. Therefore, the value of this metric also pushes the prediction to the right. Finally, the density of comment lines of this class is 17.15%, a value much lower than the average of our dataset (39% according to Table 4 of the TSE paper). We saw that lower values of the “comment lines density” metric have a moderate and positive impact on labeling a class as high-TD. Therefore, the value of this metric also pushes the prediction to the right.


On the other hand, “wmc”, “max nested blocks”, “lcom” etc. all have a negative impact on labeling the class as high-TD. The value of wmc for this class is 12, which is lower than the average value of our dataset (17.5 according to Table 4 of the TSE paper). We also saw that lower values of the “wmc” metric have a negative impact on labeling a class as high-TD. Therefore, probably because of its low value, this metric pushes the prediction to the left. The same holds for max nested blocks with a value of 12 (average value of our dataset is 17.5 according to Table 4 of the TSE paper), lcom with a value of 9 (average value of our dataset is 63.58 according to Table 4 of the TSE paper), and cbo with a value of 5 (average value of our dataset is 7.44 according to Table 4 of the TSE paper). We saw that lower values of the these metrics have a negative impact on labeling a class as high-TD. Therefore, these predictor also contribute towards labeling this class as not high-TD.

This class is labeled as not high-TD with a probability of 0.02 (below 0.5).

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][11116,:], features=X.iloc[11116,:])

In another single prediction example, “average code churn” and “hunks count” have a positive impact on labeling the class as high-TD. The average size of a code churn of this class is 104, a value much higher than the average of our dataset (27.31 according to Table 4 of the TSE paper). Therefore, the value of this metric pushes the prediction to the right. Moreover, the median number of hunks made to this class in the evolution period is 3, a value higher than the average of our dataset (1.76 according to Table 4 of the TSE paper). Therefore, probably because of its high value, this metric also pushes the prediction to the right. 

On the other hand, “wmc”, “comments lines density”, “duplicated lines density” etc. all have a negative impact on labeling the class as high-TD. The value of wmc of this class is 17, which is lower than the average value of our dataset (17.5 according to Table 4 of the TSE paper). Therefore, probably because of its low value, this metric pushes the prediction to the left. The same holds for the density of comment lines and duplicated blocks. Regarding “comments lines density”, the value of this metric for this class is 55.72%, which is much higher than the corresponding averaged value of our dataset (39% according to Table 4 of the TSE paper). Regarding “duplicated lines density”, it seems that this class has 0 duplications. Therefore, both of these predictors push the prediction to the left, along with many other metrics.

This class is labeled as not high-TD with a probability of 0.01 (below 0.5).

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][3730,:], features=X.iloc[3730,:])

In another single prediction example, “wmc” and “total refactorings” (among others) have a positive impact on labeling this class as high-TD and push the prediction to the right, probably because of their high values. On the other hand, “comment lines density” (probably because of its high value), and “max nested blocks” (probably because of its low value) push the prediction to the left, along with many other metrics. This particular class is labeled as not high-TD with a probability of 0.08 (below 0.5).

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][15,:], features=X.iloc[15,:])

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][16189,:], features=X.iloc[16189,:])

In another single prediction example, “wmc”, “total refactorings” and “max nested blocks” have a positive impact on labeling this class as high-TD and push the prediction to the right, probably because of their high values. On the other hand, “lcom”, “contributors count” (probably because of their low values) and “comment lines density” (probably because of its high value) push the prediction to the left. This particular class is labeled as not high-TD with a probability of 0.21 (below 0.5).

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][2,:], features=X.iloc[2,:])

In another single prediction example, “wmc”, “max nested blocks”, "lcom", and "total refactorings" (among others) have a positive impact on labeling this class as high-TD and push the prediction far to the right, probably because of their extremely high values compared to the average values of these metrics for our dataset (17.5, 1.31, 63.58, and 15.66 respectively). In this example, the metrics that contribute towards labelling the class as high-TD have overshadowed the metrics that push the prediction to the left. This particular class is labeled as high-TD with a probability of 1.00 (well above 0.5).

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][3236,:], features=X.iloc[3236,:])

Finally, in the above single prediction example, high values of “wmc”, “total refactorings”, “lcom”, and “cbo” have a strong positive impact on labeling this class as high-TD and push the prediction far to the right. This particular class is labeled as high-TD with a probability of 0.87 (well above 0.5).

## Other

In [ ]:
shap.dependence_plot('contributors_count', shap_values[1], features=X, interaction_index='contributors_experience')

In [ ]:
shap.decision_plot(explainer.expected_value[1], shap_values[1][2,:], features=X.iloc[2,:])